In [ ]:
%pip install sentence-transformers

In [ ]:
from io import BytesIO
import base64
from pathlib import Path
import pandas as pd
import numpy as np
from transformers import ViTImageProcessor, ViTModel, ViTFeatureExtractor
from PIL import Image
import torch
import requests
import tqdm as tqdm
from sentence_transformers import SentenceTransformer, util
from PIL import Image, ImageFile
import requests
import torch
import json
# pip install accelerate
import torch
import requests
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration


In [ ]:
processor = Blip2Processor.from_pretrained("Gregor/mblip-mt0-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Gregor/mblip-mt0-xl", torch_dtype=torch.bfloat16, device_map="auto")

In [ ]:
print('Loaded image')

In [ ]:

# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

question = "Describe the image in Russian."
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.bfloat16)

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

In [ ]:
df = pd.read_csv('/kaggle/input/nto-cv-olympiad-final-dataset/merged/merged.csv')

In [ ]:
def img_from_base64(data: str) -> Image.Image:
    return Image.open(BytesIO(base64.b64decode(data, validate=False)))

In [ ]:
images = list(df.image.apply(img_from_base64))

In [ ]:
BATCH_SIZE = 128

In [ ]:
result = []
for i in tqdm.trange(0, len(images), BATCH_SIZE):
    img_batch = images[i:i+BATCH_SIZE]
    inputs = processor(img_batch, [question] * len(img_batch), return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs)
    print(f'Processing images {i} to {i + BATCH_SIZE}')
    for i in range(len(img_batch)):
        result.append(processor.decode(outputs[i], skip_special_tokens=True))

In [ ]:
pd.DataFrame({'XID': df.XID, 'text': result}).to_csv('descriptions.csv', index=False)